In [1]:
%load_ext autoreload
%autoreload 2
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas

Thu Dec 06 2018 

CPython 3.6.6
IPython 6.5.0

numpy 1.15.1
scipy 1.1.0
sklearn 0.19.1
pandas 0.23.4

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.9.0-7-amd64
machine    : x86_64
processor  : 
CPU cores  : 12
interpreter: 64bit


In [2]:
%matplotlib inline
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import os
import nolds
import data
import mne
import matplotlib.pyplot as plt

PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
DATA_ROOT = os.path.abspath(os.path.join(PROJ_ROOT, 'data'))
PROCESSED_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'processed'))
RAW_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'raw'))
LABELED_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'labeled'))
DURATIONS_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'durations'))
sys.path.append(os.path.join(PROJ_ROOT, 'src'))
CHANNEL_NAMES = ['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
                 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
META_COLUMN_NAMES = ['freq', 'RESP_4W', 'RESP_FIN', 'REMISE_FIN', 'AGE', 'SEX', 'M_1',
       'M_4', 'M_F', 'délka léčby', 'lék 1', 'lék 2', 'lék 3', 'lék 4']
META_FILE_NAME = 'DEP-POOL_Final_144.xlsx'
meta_df = pd.read_excel(os.path.join(RAW_ROOT, META_FILE_NAME), index_col='ID', names=META_COLUMN_NAMES)

raw_fif = mne.io.read_raw_fif(os.path.join(PROCESSED_ROOT, '1a.fif'))
t = pd.DataFrame(raw_fif.get_data())
data = pd.DataFrame(np.transpose(t.values), columns=CHANNEL_NAMES)
data = np.transpose(data.values)

Opening raw data file /home/kovar/thesis_project/data/processed/1a.fif...
This filename (/home/kovar/thesis_project/data/processed/1a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
Isotrak not found
    Range : 0 ... 19104 =      0.000 ...    76.416 secs
Ready.


<ipython-input-2-cc1fabbf0faf>:26: RuntimeWarning: This filename (/home/kovar/thesis_project/data/processed/1a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  raw_fif = mne.io.read_raw_fif(os.path.join(PROCESSED_ROOT, '1a.fif'))


In [22]:
from lib.nolitsa.nolitsa import utils

def find_least_stationary_window(x, win_width=5000, slide_width=250):
    assert len(x) > win_width
    start, end, min_start, min_end = 0, win_width, 0, 0
    min_p_value = np.inf
    
    iters = 0
    while end < len(x):
        w = x[start:end]
        p_value = utils.statcheck(w)[1]
        if p_value < min_p_value:
            min_p_value = p_value
            min_start, min_end = start, end
        start += slide_width
        end += slide_width
        
    print('Min. p-value: {}'.format(min_p_value))
    return x[min_start:min_end]

In [23]:
windows = []
for chan_data in data:
    windows.append(find_least_stationary_window(chan_data))

Min. p-value: 0.11369420028956398
Min. p-value: 0.004414317794923996
Min. p-value: 0.7916910682849563
Min. p-value: 0.5553762634557045
Min. p-value: 0.7634929758455851
Min. p-value: 0.07518012947110826
Min. p-value: 0.7783474946812966
Min. p-value: 0.0014208299453373388
Min. p-value: 0.8836628546519442
Min. p-value: 0.9840377592383684
Min. p-value: 1.563153183219955e-25
Min. p-value: 0.0001512935341372778
Min. p-value: 0.7506483406994664
Min. p-value: 0.6423119137428049
Min. p-value: 0.6715190508479489
Min. p-value: 0.9820326156433871
Min. p-value: 0.0002926996293150913
Min. p-value: 0.7160425252987777
Min. p-value: 0.35041457293280487


In [ ]:
500